In [5]:
import os
os.listdir('/Users/derekdewald/Documents/Python/Github_Repo/automated_py_jobs/daily_job.py')

['daily_job.py']

In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from daily_processes import generate_dictionary
d_outline,examine_further  = generate_dictionary()

In [ ]:
######################################

# Action Add DQ Item.
# Add Read Me, which Explanation of process and Definitions of all Categorizations.

# DQ Validation Component


# Question I need to ask Myself.
# Is it a Definition.
# Is it a Step in a Process
# Is it a Process
# Is it information Related to a Process.

######################################

In [ ]:
# DQ Process 1: Need to Avoid Duplication of Word in ALL SHEETS.

In [ ]:
# Definitions: Are Technical Definitions, descriptions and eloborate Notes describing a Term. 

# Notes: Merge in Definitions.
# Take Longer Listings and organize them, Models, BLUE, 

In [96]:
# Iteratively Distribute all of Definitions into Notes.

# Create Merge Flags into DataFrame
# Original Test was to merge on Process and Word. However it was clear that there was no benefit to this, other than to avoid DQ ISSUES if
# a straight Merge on Word was to be used, which it was very valuable and simple. 
notes = notes_df.copy()
definition = definition_df1.copy() 



step1 = ['Word']
temp_df = definition.merge(notes[step1],on=step1,how='left',indicator=True).rename(columns={'_merge':'ON_WORD'}).drop_duplicates().reset_index(drop=True)

# Need to ensure Order when merging into Notes.

notes


# Can we Apply Definitoins to Process or Categorization. Distribute what we've already matched.
#temp_df = 

,Process,Categorization,Word,Definition,ON_WORD
0,Mathematics,Concept,Homoscedasticity,Variance of the residuals should be constant.,both
1,Mathematics,Concept,Independence,Observations should be independent of each oth...,both
2,Mathematics,Concept,Linearity,Relationship between Independent and Dependent...,both
3,Mathematics,Property,Colinearity,No perfect linear relationship between residuals,left_only
4,Mathematics,Concept,Normality,Residuals should be normally distributed.,left_only
5,Mathematics,Theorem,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...,left_only
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...,both
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing...",both
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...,both
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...,both


,Process,Categorization,Word,Definition
0,Mathematics,Concept,Homoscedasticity,Variance of the residuals should be constant.
1,Mathematics,Concept,Independence,Observations should be independent of each oth...
2,Mathematics,Concept,Linearity,Relationship between Independent and Dependent...
3,Mathematics,Property,Colinearity,No perfect linear relationship between residuals
4,Mathematics,Concept,Normality,Residuals should be normally distributed.
5,Mathematics,Theorem,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing..."
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...


In [70]:
from data_d_dicts import links

notes_df = pd.read_csv(links['google_notes_csv'])
notes_df

,Process,Categorization,Word,Definition
0,ML Project,Process Step,Problem Definition,"Define, Quantify and Articulate Problem."
1,ML Project,Process Step,Problem Definition,Project Goal Explicitly stated in 3-4 Sentence...
2,ML Project,Process Step,Problem Definition,Explicilty Defined Business Delieverables
3,ML Project,Process Step,Problem Definition,Explicilty Defined Expected Outcomes and KPIs
4,ML Project,Process Step,Problem Definition,"Identify Model Baseline, or who is responsible..."
...,...,...,...,...
66,Best Linear Unbiased Estimator,Requirement,Homoscedasticity,NaN
67,Best Linear Unbiased Estimator,Requirement,Independence,NaN
68,Best Linear Unbiased Estimator,Requirement,Linearity,NaN
69,Best Linear Unbiased Estimator,Requirement,No Perfect Collinearity,NaN


In [75]:
notes_df[notes_df['Process']=='ML Project']

,Process,Categorization,Word,Definition
0,ML Project,Process Step,Problem Definition,"Define, Quantify and Articulate Problem."
1,ML Project,Process Step,Problem Definition,Project Goal Explicitly stated in 3-4 Sentence...
2,ML Project,Process Step,Problem Definition,Explicilty Defined Business Delieverables
3,ML Project,Process Step,Problem Definition,Explicilty Defined Expected Outcomes and KPIs
4,ML Project,Process Step,Problem Definition,"Identify Model Baseline, or who is responsible..."
5,ML Project,Process Step,Data Collection,Need to Create Definition and Requirements
6,ML Project,Process Step,Data Preparation,Identify Data Elements which appear relevant (...
7,ML Project,Process Step,Data Preparation,"Label, and Document all data elements in Model..."
8,ML Project,Process Step,Data Preparation,"Document data omissions, known data quality is..."
9,ML Project,Process Step,Data Preparation,"Document all Imputation, Deletion, Standardiza..."


In [74]:
notes_df.Process.unique().tolist()

['ML Project',
 'ML Model',
 'Problem Solving',
 'Process Development',
 'Communication',
 'Goals',
 'A|B Testing',
 'Best Linear Unbiased Estimator']

,Word,Definition
0,Homoscedasticity,Variance of the residuals should be constant.
1,Independence,Observations should be independent of each oth...
2,Linearity,Relationship between Independent and Dependent...
3,Colinearity,No perfect linear relationship between residuals
4,Normality,Residuals should be normally distributed.
...,...,...
321,Contextual Data,Provides situational or external information t...
322,Structural Data,Represents relatively stable attributes that d...
323,Semantic Type,Influenced by the literal definition of semant...
324,Functional Role,Describes how a variable is used within a spec...


1. Process
2. Categorization
3. Word
4. Definition

In [53]:
notes_df[word].unique().tolist()

['Process Step', 'Consideration', 'Guiding Principle', 'Requirement']

In [57]:
final_df = pd.DataFrame(['Process','Categorization','Word','Definition'],columns=['Word'])
final_df['Process'] = 'Google Sheets'
final_df['Categorization'] = 'Read Me'
final_df


,Word,Process,Categorization
0,Process,Google Sheets,Read Me
1,Categorization,Google Sheets,Read Me
2,Word,Google Sheets,Read Me
3,Definition,Google Sheets,Read Me


In [52]:
for word in ['Process','Categorization']:
    pass
    
    

In [35]:
# Step 1 Prepare D Definitions so they can be concatentated into D Notes
# Step 2 Determine Filtering Logic - Based on Original Position and perhpas some generic Rules.


# Step 1 = Restructure all Categorization Records which share a value with Word
word_list = df1['Word'].unique().tolist()
categorization_list

# Items which will be REMOVED
temp_df = df1[df1['Categorization'].isin(word_list)]

# DF Which will remain
final_df = df1[~df1['Categorization'].isin(word_list)]

final_df


,Category,Categorization,Word,Definition
5,Mathematics,Theorm,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing..."
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...
10,ML Project,Pipeline Stage,Evaluation,Evaluation measures how well the final model p...
11,ML Project,Pipeline Stage,Feature Engineering,Feature engineering is the process of creating...
12,ML Project,Pipeline Stage,Feature Selection,Feature selection involves choosing a subset o...
13,ML Project,Pipeline Stage,Hyperparameter Tuning,Hyperparameter tuning involves selecting optim...
14,Modelling,Model Architecture,Autoregressive,Autoregressive models generate outputs one ste...


,Category,Categorization,Word,Definition
0,Requirement,Best Linear Unbiased Estimator,Homoscedasticity,Variance of the residuals should be constant.
1,Requirement,Best Linear Unbiased Estimator,Independence,Observations should be independent of each oth...
2,Requirement,Best Linear Unbiased Estimator,Linearity,Relationship between Independent and Dependent...
3,Requirement,Best Linear Unbiased Estimator,No Perfect Collinearity,No perfect linear relationship between residuals
4,Requirement,Best Linear Unbiased Estimator,Normality of Residuals,Residuals should be normally distributed.
5,Mathematics,Theorm,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing..."
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...


In [27]:



unique_dict = {}
# Create a Unique List Dict for Each of Primary
for word in primary:
    if word != 'Definition':
        unique_dict[word] = df1[word].unique().tolist()

# Test if this word appears in OTHER Columns

for word in primary:
    if word !='Definition':
        for key in unique_dict.keys():
            if key!=word:
                df1[f'{word}_{key}'] = np.where(df1[word].isin(unique_dict[key]),1,0)
                
df1.head()


,Category,Categorization,Word,Definition,Category_Categorization,Category_Word,Categorization_Category,Categorization_Word,Word_Category,Word_Categorization
0,Requirement,Best Linear Unbiased Estimator,Homoscedasticity,Variance of the residuals should be constant.,0,0,0,1,0,0
1,Requirement,Best Linear Unbiased Estimator,Independence,Observations should be independent of each oth...,0,0,0,1,0,0
2,Requirement,Best Linear Unbiased Estimator,Linearity,Relationship between Independent and Dependent...,0,0,0,1,0,0
3,Requirement,Best Linear Unbiased Estimator,No Perfect Collinearity,No perfect linear relationship between residuals,0,0,0,1,0,0
4,Requirement,Best Linear Unbiased Estimator,Normality of Residuals,Residuals should be normally distributed.,0,0,0,1,0,0


In [28]:
display(df1)

,Category,Categorization,Word,Definition,Category_Categorization,Category_Word,Categorization_Category,Categorization_Word,Word_Category,Word_Categorization
0,Requirement,Best Linear Unbiased Estimator,Homoscedasticity,Variance of the residuals should be constant.,0,0,0,1,0,0
1,Requirement,Best Linear Unbiased Estimator,Independence,Observations should be independent of each oth...,0,0,0,1,0,0
2,Requirement,Best Linear Unbiased Estimator,Linearity,Relationship between Independent and Dependent...,0,0,0,1,0,0
3,Requirement,Best Linear Unbiased Estimator,No Perfect Collinearity,No perfect linear relationship between residuals,0,0,0,1,0,0
4,Requirement,Best Linear Unbiased Estimator,Normality of Residuals,Residuals should be normally distributed.,0,0,0,1,0,0
5,Mathematics,Theorm,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...,0,0,0,0,0,1
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...,0,0,0,0,0,0
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing...",0,0,0,0,0,0
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...,0,0,0,0,0,0
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...,0,0,0,0,0,0


In [25]:
df1

,Category,Categorization,Word,Definition,in_Categorization,in_Word,in_Category
0,Requirement,Best Linear Unbiased Estimator,Homoscedasticity,Variance of the residuals should be constant.,0,1,0
1,Requirement,Best Linear Unbiased Estimator,Independence,Observations should be independent of each oth...,0,1,0
2,Requirement,Best Linear Unbiased Estimator,Linearity,Relationship between Independent and Dependent...,0,1,0
3,Requirement,Best Linear Unbiased Estimator,No Perfect Collinearity,No perfect linear relationship between residuals,0,1,0
4,Requirement,Best Linear Unbiased Estimator,Normality of Residuals,Residuals should be normally distributed.,0,1,0
5,Mathematics,Theorm,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...,1,0,0
6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...,0,0,0
7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing...",0,0,0
8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...,0,0,0
9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...,0,0,0


In [ ]:
question_to_gpt = 'Can you provide me a 3-4 Sentence definition highlighting the definition, its origin, its importance and its application. If it is a model can you define >>>>>>'

In [ ]:
def generate_read_me():

    from data_d_dicts import links
    df = pd.read_csv(links['google_definition_csv'])

    read_me_df = pd.DataFrame(df.columns,columns=['W'])

    return read_me_df
    

In [142]:
df.head(1)

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df.merge(df[['Category','Categorization','Definition','Word']],on='Word',how='left')

,Word,Category,Categorization,Definition
0,Category,NaN,NaN,NaN
1,Categorization,NaN,NaN,NaN
2,Word,NaN,NaN,NaN
3,Definition,NaN,NaN,NaN
4,Notes,NaN,NaN,NaN
5,Link,NaN,NaN,NaN
6,Image,NaN,NaN,NaN
7,Markdown Equation,NaN,NaN,NaN
8,Dataset Size,NaN,NaN,NaN
9,Learning Type,NaN,NaN,NaN


In [138]:
read_me_df = pd.DataFrame(df.columns,columns=['Word'])
read_me_df


,Dataset Columns
0,Category
1,Categorization
2,Word
3,Definition
4,Notes
5,Link
6,Image
7,Markdown Equation
8,Dataset Size
9,Learning Type


In [161]:
df1['L1'] = 

,index,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,0,Requirement,Best Linear Unbiased Estimator,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Requirement,Best Linear Unbiased Estimator,Independence,Observations should be independent of each oth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Requirement,Best Linear Unbiased Estimator,Linearity,Relationship between Independent and Dependent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Requirement,Best Linear Unbiased Estimator,No Perfect Collinearity,No perfect linear relationship between residuals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Requirement,Best Linear Unbiased Estimator,Normality of Residuals,Residuals should be normally distributed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Mathematics,Theorm,Best Linear Unbiased Estimator,The Best Linear Unbiased Estimator (BLUE) is a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,ML Project,Pipeline Stage,"Bias, Fairness and Ethics",This step examines whether the model produces ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,ML Project,Pipeline Stage,Data Collection,"Data collection involves identifying, sourcing...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,ML Project,Pipeline Stage,Data Preparation,Data preparation focuses on cleaning and struc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,ML Project,Pipeline Stage,Deployment,Deployment is the process of integrating the t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
from data_d_dicts import links

df = pd.read_csv(links['google_definition_csv'])
df

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Best Linear Unbiased Estimator,Requirements,Independence,Observations should be independent of each oth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity,No perfect linear relationship between residuals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Data Preparation,Functional Role,Contextual Data,Provides situational or external information t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,Data Preparation,Functional Role,Structural Data,Represents relatively stable attributes that d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,Machine Learning,Definition,Semantic Type,Influenced by the literal definition of semant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,Machine Learning,Definition,Functional Role,Describes how a variable is used within a spec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
df['Categorization'].unique()

array(['Requirements', 'Definition', 'Algorithm', 'Function', 'Procedure',
       'Optimizer', 'Regularization', 'Clustering',
       'Centroid-based clustering', 'Density-based clustering',
       'Probabilistic clustering', 'Graph-based clustering',
       'Insance Based Learning', 'Evaluation', 'Semantic Type ',
       'Functional Role '], dtype=object)

In [151]:
l1 = ['Mathetmatics','Machine Learning','Organization','Tooling']
df[~df['Category'].isin(l1)]
      

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Best Linear Unbiased Estimator,Requirements,Independence,Observations should be independent of each oth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity,No perfect linear relationship between residuals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,Data Preparation,Semantic Type,Demographic,Representing member level attributes and chara...,Exclude Behavioural Clustering,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Data Preparation,Functional Role,Core Variables,Primary drivers of cluster formation and direc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,Data Preparation,Functional Role,Contextual Data,Provides situational or external information t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,Data Preparation,Functional Role,Structural Data,Represents relatively stable attributes that d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Best Linear Unbiased Estimator,Requirements,Independence,Observations should be independent of each oth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity,No perfect linear relationship between residuals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Data Preparation,Functional Role,Contextual Data,Provides situational or external information t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,Data Preparation,Functional Role,Structural Data,Represents relatively stable attributes that d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,Machine Learning,Definition,Semantic Type,Influenced by the literal definition of semant...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,Machine Learning,Definition,Functional Role,Describes how a variable is used within a spec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
df[df['Category']=='Validation']

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
276,Validation,Function,Accuracy,Number of correctly classified examples divide...,NaN,NaN,NaN,$$ Accuracy = \frac{TP + TN}{TP + TN + FP + FN...,NaN,NaN,NaN,NaN,NaN
277,Validation,Procedure,Confusion Matrix,Table used to evaluate the performance of a cl...,NaN,NaN,https://raw.githubusercontent.com/derek-dewald...,NaN,NaN,NaN,NaN,NaN,NaN
278,Validation,Procedure,K Fold Cross Validation,Application of Cross Validation. Method for in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,Validation,Procedure,Startified Cross Validation,Increasing the Principle of Cross Validation a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
df.groupby(['Category','Categorization']).count()

Word  Definition  \
Category                       Categorization                                
Best Linear Unbiased Estimator Requirements                  5           5   
Clustering                     Centroid-based clustering     1           1   
                               Clustering                    1           1   
                               Density-based clustering      2           2   
                               Evaluation                    1           1   
                               Graph-based clustering        1           1   
                               Probabilistic clustering      1           1   
Data Preparation               Functional Role               3           3   
                               Semantic Type                 6           6   
Machine Learning               Clustering                    4           4   
                               Definition                   36          36   
                               Insance Based Learning        1           1   
Mathematics                    Definition                    5           5   
Model                          Algorithm                   201          20   
                               Function                      1           0   
                               Procedure                    19           0   
                               Regularization                3           3   
Organization                   Definition                    6           6   
Tool                           Definition                    2           2   
Training                       Function                      2           2   
                               Optimizer                     7           7   
Validation                     Function                      1           1   
                               Procedure                     3           3   

                                                          Notes  Link  Image  \
Category                       Categorization                                  
Best Linear Unbiased Estimator Requirements                   0     0      0   
Clustering                     Centroid-based clustering      0     0      0   
                               Clustering                     0     0      0   
                               Density-based clustering       0     0      0   
                               Evaluation                     0     0      0   
                               Graph-based clustering         0     0      0   
                               Probabilistic clustering       0     0      0   
Data Preparation               Functional Role                0     0      0   
                               Semantic Type                  6     0      0   
Machine Learning               Clustering                     0     0      0   
                               Definition                     0     0      3   
                               Insance Based Learning         0     0      0   
Mathematics                    Definition                     0     0      0   
Model                          Algorithm                      0     0      2   
                               Function                       0     0      0   
                               Procedure                      0     0      0   
                               Regularization                 1     0      0   
Organization                   Definition                     0     0      0   
Tool                           Definition                     0     0      0   
Training                       Function                       0     0      0   
                               Optimizer                      0     0      0   
Validation                     Function                       0     0      0   
                               Procedure                      0     0      1   

                                                          Markdown Equation  \
Category                   

In [132]:
df.head()

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class,Model,Ensemble
0,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Best Linear Unbiased Estimator,Requirements,Independence,Observations should be independent of each oth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Best Linear Unbiased Estimator,Requirements,No Perfect Collinearity,No perfect linear relationship between residuals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
for value in ['Dataset Size','Learning Type','Algorithm Class', 'Model']:
    print(f'Column Name:{value}, options:{df[value].unique().tolist()}')

Column Name:Dataset Size, options:[nan, 'Small, Medium', 'Small', 'Medium, Large', 'Large', 'Medium', 'Small, Medium, Large', 'Unknown']
Column Name:Learning Type, options:[nan, 'Supervised']
Column Name:Algorithm Class, options:[nan, 'Classification', 'Regression']
Column Name:Model, options:[nan, 'Linear', 'Tree', 'Meta-Model']


In [121]:
df['Word'].tolist()

['SGDClassifier',
 'VotingClassifier',
 'Quantile Random Forest',
 'RandomTreesEmbedding',
 'Area Under the Curve',
 'Perceptron',
 'RidgeCV',
 'Regularization',
 'SelectFpr',
 'AdaBoostClassifier']

In [120]:
file_location='/Users/derekdewald/Documents/Python/Github_Repo/Data/daily_test_results.csv'
df = pd.read_csv(file_location)
df

,Date,Action,Result,Score,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,2026-01-07,Test,Fail,-1,Model,Algorithm,SGDClassifier,Handle Large Dataset well because it treats ea...,NaN,NaN,NaN,NaN,Large,NaN,NaN
1,2026-01-07,Update,Push,0,Model,Algorithm,VotingClassifier,NaN,NaN,NaN,NaN,NaN,"Medium, Large",NaN,NaN
2,2026-01-07,Update,Push,0,Model,Algorithm,Quantile Random Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2026-01-07,Update,Push,0,Model,Algorithm,RandomTreesEmbedding,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
4,2026-01-07,Test,Fail,-1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2026-01-08,Test,Push,0,Model,Algorithm,Perceptron,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
6,2026-01-08,Test,Push,0,Model,Algorithm,RidgeCV,NaN,NaN,NaN,NaN,NaN,"Medium, Large",NaN,NaN
7,2026-01-08,Update,Fail,-1,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2026-01-08,Test,Push,0,Model,Algorithm,SelectFpr,NaN,NaN,NaN,NaN,NaN,Large,NaN,NaN
9,2026-01-08,Test,Push,0,Model,Algorithm,AdaBoostClassifier,NaN,NaN,NaN,NaN,NaN,"Small, Medium",NaN,NaN


In [118]:
review_test_results()
daily_test()

2026-01-01
Category: Model
Classification: Algorithm
Word: SGDClassifier



######################### THINK ABOUT IT!  ######################### 


Definition: Handle Large Dataset well because it treats each individual point distinctly.
Notes: nan
Link: nan
Markdown: nan
Dataset Size: Large
Learning Type: nan
Alogrithm Class: nan

Category: Machine Learning
Classification: Definition
Word: Area Under the Curve



######################### THINK ABOUT IT!  ######################### 


Definition: AUC (Area Under the Curve) measures a model’s ability to distinguish between classes by summarizing the performance of the ROC (Receiver Operating Characteristic) curve. It represents the probability that the model ranks a randomly chosen positive example higher than a randomly chosen negative one. AUC ranges from 0.5 (no better than chance) to 1.0 (perfect separation), with higher values indicating better classification performance.
AUC is primarily a binary classification metric, so when using against a multivariate challenge must determine how to score, OVR, OVO. One Vs Rest, One vs One. Using MNist as an example, OVR evaluates 10 0 vs (1,2,3,4,5,6,7,8,9), 1 vs () ... etc, Where OVO measures 45 0 Vs 1, 0 vs 2, etc..
Notes: nan
Link: nan
Markdown: nan
Dataset Size: nan
Learning Type: nan
Alogrithm Class: nan

Category: Machine Learning
Classification: Definition
Word: Regularization



######################### THINK ABOUT IT!  ######################### 


Definition: Technique used to prevent overfitting by adding a penalty term to the loss function. It discourages the model from fitting too closely to the training data, ensuring better generalization to unseen data. Traditionnaly results in bias, at benefit of materially lowering Variance. Regularization is a Constraint.
Notes: nan
Link: nan
Markdown: nan
Dataset Size: nan
Learning Type: nan
Alogrithm Class: nan



,Date,Action,Result,Score,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,2026-01-07,Test,Fail,-1,Model,Algorithm,SGDClassifier,Handle Large Dataset well because it treats ea...,NaN,NaN,NaN,NaN,Large,NaN,NaN
1,2026-01-07,Test,Fail,-1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2026-01-08,Update,Fail,-1,Machine Learning,Definition,Regularization,Technique used to prevent overfitting by addin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
